In [14]:
import pandas as pd
import pickle

dict_results = {}
for ds in ['Salinas','IndianPines']:
    for dr_method in ['PCA', 'FastICA']:
        for n_components in [8, 15]:
            for model in ['SVM', 'hamida']:
                filename = f'{ds}_{dr_method}_{n_components}_{model}_1epochs.csv'
                print('\n' + filename)
                try:
                    df = pd.read_csv(f'./Results/{ds}_{dr_method}_{n_components}_{model}_30epochs.csv', index_col=0)
                    print('Loaded.')
                    try:
                        df['Confusion matrix'] = df['Confusion matrix'].apply(lambda x: pickle.loads(eval(x)))
                        df['F1 scores'] = df['F1 scores'].apply(lambda x: pickle.loads(eval(x)))
                        dict_results[f'{ds}_{dr_method}_{n_components}_{model}'] = df
                        print('Columns unpickled.')
                    except:
                        print("Problem with pickled columns")
                except:
                    print("Couldn't find CSV file.")
                
                


Salinas_PCA_8_SVM_1epochs.csv
Couldn't find CSV file.

Salinas_PCA_8_hamida_1epochs.csv
Loaded.
Columns unpickled.

Salinas_PCA_15_SVM_1epochs.csv
Couldn't find CSV file.

Salinas_PCA_15_hamida_1epochs.csv
Loaded.
Columns unpickled.

Salinas_FastICA_8_SVM_1epochs.csv
Couldn't find CSV file.

Salinas_FastICA_8_hamida_1epochs.csv
Couldn't find CSV file.

Salinas_FastICA_15_SVM_1epochs.csv
Couldn't find CSV file.

Salinas_FastICA_15_hamida_1epochs.csv
Couldn't find CSV file.

IndianPines_PCA_8_SVM_1epochs.csv
Couldn't find CSV file.

IndianPines_PCA_8_hamida_1epochs.csv
Couldn't find CSV file.

IndianPines_PCA_15_SVM_1epochs.csv
Couldn't find CSV file.

IndianPines_PCA_15_hamida_1epochs.csv
Couldn't find CSV file.

IndianPines_FastICA_8_SVM_1epochs.csv
Couldn't find CSV file.

IndianPines_FastICA_8_hamida_1epochs.csv
Couldn't find CSV file.

IndianPines_FastICA_15_SVM_1epochs.csv
Couldn't find CSV file.

IndianPines_FastICA_15_hamida_1epochs.csv
Couldn't find CSV file.


In [2]:
SALINAS_LABELS = [
    "Undefined",
    "Brocoli_green_weeds_1",
    "Brocoli_green_weeds_2",
    "Fallow",
    "Fallow_rough_plow",
    "Fallow_smooth",
    "Stubble",
    "Celery",
    "Grapes_untrained",
    "Soil_vinyard_develop",
    "Corn_senesced_green_weeds",
    "Lettuce_romaine_4wk",
    "Lettuce_romaine_5wk",
    "Lettuce_romaine_6wk",
    "Lettuce_romaine_7wk",
    "Vinyard_untrained",
    "Vinyard_vertical_trellis",
]

In [3]:
dict_results['Salinas_PCA_15_hamida']

,Confusion matrix,Accuracy,F1 scores,Kappa
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",57.948604,"[0.0, 0.0, 0.7662163555738888, 0.0032467532467...",0.520811


In [13]:
import plotly.express as px

conf_matrix = dict_results['Salinas_PCA_15_hamida']['Confusion matrix'][0]
fig = px.imshow(conf_matrix, text_auto=True, 
                labels=dict(x='Predicted labels', y='Actual labels', color='# of classification'),
                x = SALINAS_LABELS,
                y = SALINAS_LABELS,
                height=700
                )
fig.update_xaxes(tickangle=45)
fig.show()